In [ ]:
%pip install langchain
%pip install -U langchain-community
%pip install python-docx 
%pip install chromadb
%pip install sentence-transformers
%pip install python-docx 


In [22]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import re
import requests
import os
from docx import Document
from tqdm import tqdm
import pandas as pd

In [5]:
def clean_text(text):
    """
    Функция для удаления "мусора" из текста, такого как пустые строки, таблицы,
    лишние пробелы, номера пунктов и лишние символы.
    """
    # Удаление лишних символов табуляции, переносов строк и последовательностей пробелов
    cleaned_text = re.sub(r'\n+', '\n', text)  # Удаление повторяющихся переносов строк
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Замена множественных пробелов на один пробел

    # Удаление строк, содержащих "Таблица" и другие нерелевантные технические данные
    cleaned_text = re.sub(r'(Таблица\s*\d+\.\d+|Приложение\s*\d+)', '', cleaned_text)
    
    # Удаление номеров пунктов и заголовков с номерами (например, 2.3.1, 4 ХАРАКТЕРИСТИКИ ИЗАВ)
    cleaned_text = re.sub(r'(\d+\.\d+(\.\d+)?|^\d+[\sА-Яа-яЁё ]+)', '', cleaned_text)

    # Удаление оставшихся технических фрагментов
    cleaned_text = re.sub(r'[^\w\s,.()ёЁА-Яа-я-]', '', cleaned_text)  # Удаление спецсимволов, кроме пунктуации
    
    # Удаление лишних пробелов, возникающих после чистки
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text).strip()
    
    return cleaned_text



In [6]:
def load_and_split_documents(documents_folder):
    """
    Загрузка документов из локальной директории и разбиение их на фрагменты.
    """
    # Загрузка документов с указанием кодировки utf-8
    loader = DirectoryLoader(
        documents_folder, glob="*.txt", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"}
    )
    documents = loader.load()
    
    for i in range(len(documents)):
        documents[i].page_content = clean_text(documents[i].page_content)

    # Разбиение документов на фрагменты
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)
    return texts

In [7]:
def create_vectorstore(texts, embedding_model_name):
    """
    Создание эмбеддингов и сохранение их в векторном хранилище ChromaDB.
    """
    # Создание эмбеддингов
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    
    # Создание векторного хранилища ChromaDB
    vectorstore = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory="chroma_db"
    )
    vectorstore.persist()
    return vectorstore

### Yandex GPT

In [8]:
def call_api(system_prompt: str, prompt: str):
    """
    Обращается к предобученной модели и возвращает текстовый результат.
    """    
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion" 
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key AQVNwy5Z1_keoKl1F1R6BxAXLOKCiq5h1NEzd4Dw", # апи ключ
        "x-folder-id": "b1ghupvijk4nvsbqtcv3" # id каталога
    }
    
    payload = {
        "modelUri": "gpt://b1ghupvijk4nvsbqtcv3/yandexgpt-32k/rc", # модель https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/models

        # опции
        "completionOptions": {
            "stream": False,
            "temperature": 0.1,
            "maxTokens": "2000"
        },
        
        "messages": [
            {
                "role": "system",
                "text": system_prompt
            },
            {
                "role": "user",
                "text": prompt
            }
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result['result']['alternatives'][0]['message']['text']
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

In [9]:
def call_api_tuned(system_prompt: str, prompt: str):
    """
    Обращается к дообученной на уникальном экологическом QnA-датасете модели и возвращает текстовый результат.
    """   
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion" 
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key AQVNzRH411adXH8Z85cfZRsooQWT9hxf1TL139Z9", # апи ключ
        "x-folder-id": "b1glut889d1giqggc38n" # id каталога
    }
    
    payload = {
        "modelUri": "ds://bt18aohnkrp4njfmrf13", # модель https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/models

        # опции
        "completionOptions": {
            "stream": False,
            "temperature": 0.1,
            "maxTokens": "2000"
        },
        
        "messages": [
            {
                "role": "system",
                "text": system_prompt
            },
            {
                "role": "user",
                "text": prompt
            }
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result['result']['alternatives'][0]['message']['text']
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

# Преобразование файлов

Начинаем с распознования файлов. После переходим на kaggle для ускорения работы.

In [ ]:
def convert_docx_to_txt(folder_path, folder_path_to):
    # Проверяем, существует ли папка
    if not os.path.isdir(folder_path):
        print(f"Папка {folder_path} не найдена.")
        return

    # Проходим по всем файлам в указанной папке
    for filename in os.listdir(folder_path):
        if filename.endswith('.docx'):  # Проверяем, что это .docx файл
            docx_path = os.path.join(folder_path, filename)
            txt_path = os.path.join(folder_path_to, filename.replace('.docx', '.txt'))

            # Чтение содержимого .docx файла
            doc = Document(docx_path)
            text = ""
            for paragraph in doc.paragraphs:
                text += paragraph.text + "\n"

            # Запись содержимого в .txt файл
            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                txt_file.write(text)

            print(f"Файл {filename} успешно преобразован в {txt_path}")



In [ ]:
folder_path = '../documents/put your docx here!'
folder_path_to = '../documents/processed files'
convert_docx_to_txt(folder_path, folder_path_to)

Файл Том 1 Инвентаризация Эко Агро.docx успешно преобразован в ../documents/processed files\Том 1 Инвентаризация Эко Агро.txt

Файл Том 2 ПДВ Эко Агро.docx успешно преобразован в ../documents/processed files\Том 2 ПДВ Эко Агро.txt


In [ ]:
documents_folder = '/kaggle/input/ecology-hack'  # Путь к папке с документами, то есть '../documents/processed files' куда мы созранили


# QnA

In [11]:
# Загрузка и разбиение документов
texts = load_and_split_documents(documents_folder)

In [ ]:
# Создание векторного хранилища
vectorstore = create_vectorstore(texts, "DeepPavlov/rubert-base-cased-sentence")

# Получение сабмишена

In [13]:
%pip install openpyxl
%pip install -q evaluate bert_score
%pip install --upgrade nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.1 MB/s eta 0:00:0000:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import numpy as np
import pandas as pd

In [16]:
df = pd.read_csv('/kaggle/input/train-dset/test.csv', encoding='utf-8', sep='\t', index_col='№ п/п')
df.head()

,Вопрос,Ответ,Документ
№ п/п,,,
1,"Объяснить, что такое источник выбросов, источн...",NaN,Нет
2,Указать этапы разработки проекта начиная с пол...,NaN,Нет
3,Расписать состав тома ПДВ,NaN,Нет
4,Как присваиваются номера источников выбросов п...,NaN,Нет
5,Что такое газоочистные установки? Приведите их...,NaN,Нет


In [23]:
len(df)

92

In [17]:
import evaluate
from evaluate import load
import nltk


In [18]:
meteor = evaluate.load('meteor')
bertscore = load("bertscore")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [19]:
answers = []

In [24]:
for i in range(len(df)):
    query = df.iloc[i]['Вопрос']
    
    retriever = vectorstore.as_retriever()
    docs = retriever.get_relevant_documents(query)
    # Формирование контекста из документов
    context = "\n".join([doc.page_content for doc in docs])
    
    prompt = f"""Контекст: {context}
    Вопрос: {query}
    Оцени, подходит ли данный контекст для ответа на вопрос! Если контекст является достаточным для ответа, используй его. Если нет, то не пиши в ответе об этом, а сразу отвечай на вопрос! В любом случае, предоставь максимально подробный и точный ответ на вопрос, используя всю свою экспертную информацию.
    """
    
    chat_response = call_api_tuned("Ты — эксперт в области экологии, защиты окружающей среды и экологического права. Твоя задача — помогать пользователям находить ответы на вопросы об экологии, экологических нормах, правилах и стандартах!", prompt)
    
    answers.append(chat_response)
    print(i+1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


In [25]:
df['answer'] = answers

In [26]:
answers[:2]

['Источник выбросов – это источник, в котором осуществляется образование загрязняющих веществ или смеси таких веществ в процессе хозяйственной и иной деятельности. Источник выделения – это объект, в котором происходит образование загрязняющих веществ (организованный источник, неорганизованный источник).',
 'Этапы разработки проекта начиная с получения в работу: получение в работу, разработка технического задания, разработка проектной документации, согласование проектной документации, утверждение проектной документации.']

In [27]:
score_meteor = meteor.compute(predictions=df['Вопрос'].values.tolist(), references=df['answer'].values.tolist())
print (score_meteor)

{'meteor': 0.33693326715754535}


In [28]:
score_bert = bertscore.compute(predictions=df['Вопрос'].values.tolist(), references=df['answer'].values.tolist(), lang="ru")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [29]:
print ('Presicion', np.array(score_bert['precision']).mean())
print ('Recall', np.array(score_bert['recall']).mean())
print ('F1', np.array(score_bert['f1']).mean())

Presicion 0.7856320693441059
Recall 0.7251067041702892
F1 0.750476215844569


# Сохраняем в csv

In [31]:
df_selected = df[['answer']]
df_selected.to_csv('answers.csv', sep='\t', index=True)

## Проверка example.csv

In [32]:
try:
    df = pd.read_csv('answers.csv', delimiter='\t')
except pd.errors.ParserError:
    print("Ошибка: Не удалось разобрать CSV-файл. Убедитесь, что разделитель - tab.")

column_name = 'answer'
if column_name not in df.columns:
    print(f"Столбец '{column_name}' не найден в таблице.")
else:
    if len(df[column_name]) != 92:
        print(f"Длина столбца '{column_name}' не совпадает с длиной таблицы.")

In [33]:
try:
    df = pd.read_csv('/kaggle/working/answers.csv', delimiter='\t')
except pd.errors.ParserError:
    print("Ошибка: Не удалось разобрать CSV-файл. Убедитесь, что разделитель - tab.")

column_name = 'answer'
if column_name not in df.columns:
    print(f"Столбец '{column_name}' не найден в таблице.")
else:
    if len(df[column_name]) != 92:
        print(f"Длина столбца '{column_name}' не совпадает с длиной таблицы.")

In [35]:
df

,№ п/п,answer
0,1,"Источник выбросов – это источник, в котором ос..."
1,2,Этапы разработки проекта начиная с получения в...
2,3,"Состав тома ПДВ: титульный лист, содержание, п..."
3,4,Номера источников выбросов присваиваются предп...
4,5,"Газоочистные установки (ГОУ) — это сооружения,..."
...,...,...
87,88,"Источники выбросов, имеющие произвольную форму..."
88,89,Жидкие и газообразные загрязняющие вещества от...
89,90,В интернете есть много сайтов с информацией на...
90,91,"Да, хлопковая пыль создаётся."
